In [1]:
# Importing required libraries

import quandl, math
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Extracting the data using the quandl API

# get the token by creating an account on quandl.com
token = '***********'

df = quandl.get("EOD/AAPL", authtoken = token)
df.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2013-09-03,493.10,500.60,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0
2013-09-04,499.56,502.24,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0
2013-09-05,500.25,500.68,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0
2013-09-06,498.44,499.38,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0
2013-09-09,505.00,507.92,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0


In [3]:
df.shape

(1090, 12)

In [4]:
# Computing new columns

df1 = df.copy()
df1['HL_PCT'] = (df1['Adj_High'] - df1['Adj_Low'])/ df1['Adj_Close'] * 100.0
df1['PCT_change'] = (df1['Adj_Close'] - df1['Adj_Open']) / df1['Adj_Open'] * 100.0

df1.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,HL_PCT,PCT_change
Date,,,,,,,,,,,,,,
2013-09-03,493.10,500.60,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0,2.711941,-0.916650
2013-09-04,499.56,502.24,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0,1.195129,-0.173953
2013-09-05,500.25,500.68,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0,1.421447,-0.995502
2013-09-06,498.44,499.38,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0,1.892738,-0.044138
2013-09-09,505.00,507.92,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0,0.877176,0.231683


In [5]:
# Feature Selection

df2 = df1[['Adj_Close','PCT_change','HL_PCT','Adj_Volume']]
df2.head()

,Adj_Close,PCT_change,HL_PCT,Adj_Volume
Date,,,,
2013-09-03,15.476472,-0.916650,2.711941,331928800.0
2013-09-04,15.796753,-0.173953,1.195129,345032800.0
2013-09-05,15.688388,-0.995502,1.421447,236367600.0
2013-09-06,15.781833,-0.044138,1.892738,359525600.0
2013-09-09,16.033661,0.231683,0.877176,340687200.0


In [6]:
# Adding the target variable 

#forecast_col = 'Adj_Close'
df2.fillna(value = -99999, inplace = True)
forecast_out = int(math.ceil(0.01 * len(df2)))

In [7]:
df2['label'] = df2['Adj_Close'].shift(-forecast_out)

In [8]:
# Summary of Data

df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1090 entries, 2013-09-03 to 2017-12-28
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Adj_Close   1090 non-null   float64
 1   PCT_change  1090 non-null   float64
 2   HL_PCT      1090 non-null   float64
 3   Adj_Volume  1090 non-null   float64
 4   label       1079 non-null   float64
dtypes: float64(5)
memory usage: 51.1 KB


In [9]:
df2.head(15)

,Adj_Close,PCT_change,HL_PCT,Adj_Volume,label
Date,,,,,
2013-09-03,15.476472,-0.916650,2.711941,331928800.0,14.719406
2013-09-04,15.796753,-0.173953,1.195129,345032800.0,14.960780
2013-09-05,15.688388,-0.995502,1.421447,236367600.0,14.805882
2013-09-06,15.781833,-0.044138,1.892738,359525600.0,15.541726
2013-09-09,16.033661,0.231683,0.877176,340687200.0,15.492944
2013-09-10,15.668432,-2.283682,3.628902,743195600.0,15.253154
2013-09-11,14.815385,0.149868,1.898612,898696400.0,15.401716
2013-09-12,14.973134,0.894344,1.986503,404051200.0,15.291799
2013-09-13,14.726374,-0.946051,1.533491,298835600.0,15.101740


In [10]:
# Checking for Null Values

a = df2.isnull().sum()
a[a > 0]

label    11
dtype: int64

In [11]:
# Dropping the rows with Null Values

df2.dropna(inplace = True)
df2.isnull().sum()

Adj_Close     0
PCT_change    0
HL_PCT        0
Adj_Volume    0
label         0
dtype: int64

In [12]:
# Splitting data into X & y

X = np.array(df2.drop(['label'], axis = 1))
y = np.array(df2['label'])

In [13]:
# Scaling the Data

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X)
X_scale = sc.transform(X)

In [14]:
# Train Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 10)

In [15]:
# Building the Linear Regression Model

from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression(n_jobs = -1)
lr_reg.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [16]:
# Intercept Value

lr_reg.intercept_

26.467977138498934

In [17]:
# Coefficients Values

print(list(zip(df2.columns[:-1], lr_reg.coef_)))

[('Adj_Close', 6.394183037257519), ('PCT_change', -0.036909559610210384), ('HL_PCT', 0.0038012272357084856), ('Adj_Volume', -0.07334893548842251)]


In [18]:
# Model Evaluation

from sklearn.metrics import mean_squared_error, r2_score

y_pred = lr_reg.predict(X_test)
print('R Square is',round(r2_score(y_pred, y_test),4))
print('MSE is',round(mean_squared_error(y_pred, y_test),4))

R Square is 0.9658
MSE is 1.3985
